# Use this notebook to serialize/export models to ONNX

In [1]:
import pandas as pd
import numpy as np

# load the results
df = pd.read_csv(
    "results.txt", 
    quotechar="'",
    index_col=False, 
    delim_whitespace=True,
    usecols=['Sport', 'Service', 'Style', 'Type', 'y', 'ModelType', 'Params'],
)

# drop everything after the seperator
seperator_idx = np.where(df['Sport'].str.startswith('*'))[0][0]
df = df.iloc[:seperator_idx]

with pd.option_context('display.max_rows', 1000, 'display.max_columns', 100, 'display.max_colwidth', 100):
    display(df)

,Sport,Service,Type,Style,y,ModelType,Params
0,nhl,draftkings,FIFTY_FIFTY,classic,top,tpot,"{""generations"": 100, ""early_stop"": 10, ""population_size"": 100, ""n_jobs"": 3}"
1,nhl,draftkings,FIFTY_FIFTY,classic,last,skautoml-pca,"{""train_time"": 600, ""per_run_time"": 120, ""model_cols"": null, ""n_components"": 5}"
2,nhl,draftkings,GPP,classic,top,skautoml,"{""train_time"": 600, ""per_run_time"": 120, ""model_cols"": null}"
3,nhl,draftkings,GPP,classic,last,skautoml-pca,"{""train_time"": 120, ""per_run_time"": 30, ""model_cols"": null, ""n_components"": 5}"
4,nhl,fanduel,GPP,classic,top,tpot,"{""generations"": 100, ""early_stop"": 10, ""population_size"": 100, ""n_jobs"": 3}"
5,nhl,fanduel,FIFTY_FIFTY,classic,top,skautoml,"{""train_time"": 600, ""per_run_time"": 120, ""model_cols"": null}"
6,nhl,fanduel,FIFTY_FIFTY,classic,last,skautoml,"{""train_time"": 600, ""per_run_time"": 120, ""model_cols"": null}"
7,nhl,yahoo,FIFTY_FIFTY,classic,top,skautoml,"{""train_time"": 600, ""per_run_time"": 120, ""model_cols"": ""best-possible-score""}"
8,nhl,yahoo,GPP,classic,last,tpot,"{""generations"": 100, ""early_stop"": 10, ""population_size"": 100, ""n_jobs"": 3}"
9,nhl,yahoo,GPP,classic,top,tpot-pca,"{""n_components"": 5, ""generations"": 100, ""early_stop"": 10, ""population_size"": 100, ""n_jobs"": 3}"


In [14]:
import json

from skl2onnx import convert_sklearn
import numpy as np
from skl2onnx.common.data_types import FloatTensorType

import fantasy_py.lineup.strategy
from fantasy_py import ContestStyle, CLSRegistry, CONTEST_DOMAIN

from automl import create_automl_model, error_report, get_df_types
from generate_train_test import generate_train_test, load_csv


PCA_COMPONENTS = 5


def func(sport, service, style: ContestStyle, contest_type, model_type: str, params):
    contest_style = ContestStyle[style.upper()]
    contest_type = CLSRegistry.get_class(CONTEST_DOMAIN, contest_type)
    full_model_name = f'{sport}_{service}_{contest_style}_{contest_type}_{model_type}'
    print(f"Exporting model '{full_model_name}'")
    display(params)
    data_df = load_csv(sport, service, contest_style, contest_type)

    model_def = json.loads(params)
    # display(model_def)
    (X_train, X_test, y_top_train, y_top_test,
     y_last_win_train, y_last_win_test) = generate_train_test(
        data_df,
        model_cols=model_def.get('model_cols'),
        random_state=5,
    )

    if model_type.startswith('automl'):
        create_model_params = {
            'framework': 'skautoml',
            'train_time': model_def.get('train_time'),
            'per_run_time_limit': model_def.get('per_run_time'),
            'overwrite': True,
        }
        if model_type.endswith('-pca'):
            create_model_params['pca_components'] = PCA_COMPONENTS,
    elif model_type == 'tpot':
        create_model_params = {
            'framework': 'tpot',
        }
    else:
        raise ValueError(f"Don't know how to process model type {model_type}")

    if row.y == 'top':
        y_train = y_top_train
        y_test = y_top_test
        model_name = 'top-score'
    elif row.y == 'last':
        y_train = y_last_win_train
        y_test = y_last_win_test
        model_name = 'last-win-score'
    else:
        raise ValueError(f"Unexpected y of {row.y}")

    sk_model, fit_params = create_automl_model(
        model_name,
        seed=1,
        **create_model_params,
    )
    print("Training model...")
    sk_model.fit(X_train, y_train, **fit_params)
    error_report(sk_model, X_test, y_test,
                 full_model_name + f": model_cols={model_def['model_cols']}")

    print(f"Exporting model to {full_model_name}.onnx")
    df_types = get_df_types(X_train)
    display(df_types)
    onnx_model = convert_sklearn(sk_model, model_name,
                                 df_types,
                                 final_types=[('variable1', FloatTensorType([1, 1]))])
    with open(full_model_name + ".onnx", "wb") as f:
        f.write(onnx_model.SerializeToString())


for _, row in df.iterrows():
    func(row.Sport, row.Service, row.Style,
         row.Type, row.ModelType, row.Params)
    break


print("Done!")


Exporting model 'nhl_draftkings_classic_<class 'fantasy_py.lineup.strategy.bet_lineup.FiftyFifty'>_tpot'


'{"generations": 100, "early_stop": 10, "population_size": 100, "n_jobs": 3}'

loading filename='nhl-draftkings-CLASSIC-FIFTY_FIFTY.csv'
127 rows of data loaded
dropping 4 rows due to nan_slate_rows=2 nan_best_score_rows=2
Training model...


KeyError: 'model_cols'